In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
import sklearn
import datetime
from statsmodels.tsa.stattools import coint

In [ ]:
import math
from tabulate import tabulate

In [ ]:
prices = pd.read_csv("C:\\Users\\vedu2\\Downloads\\Prices.csv")

In [ ]:
prices.head()

,date,PX_LAST,PX_VOLUME,Ticker,Name
0,01-01-2008,407.80,31627.0,EIM IS Equity,Eicher Motors Ltd
1,02-01-2008,404.10,38627.0,EIM IS Equity,Eicher Motors Ltd
2,03-01-2008,425.85,320648.0,EIM IS Equity,Eicher Motors Ltd
3,04-01-2008,415.50,72443.0,EIM IS Equity,Eicher Motors Ltd
4,07-01-2008,414.85,93726.0,EIM IS Equity,Eicher Motors Ltd


In [ ]:
prices.drop(["Name"],axis=1,inplace=True)

In [ ]:
prices['date'] = pd.to_datetime(prices['date'],dayfirst = True)


In [ ]:
prices.head()

,date,PX_LAST,PX_VOLUME,Ticker
0,2008-01-01,407.80,31627.0,EIM IS Equity
1,2008-01-02,404.10,38627.0,EIM IS Equity
2,2008-01-03,425.85,320648.0,EIM IS Equity
3,2008-01-04,415.50,72443.0,EIM IS Equity
4,2008-01-07,414.85,93726.0,EIM IS Equity


In [ ]:
df = prices
df.head()

,date,PX_LAST,PX_VOLUME,Ticker
0,2008-01-01,407.80,31627.0,EIM IS Equity
1,2008-01-02,404.10,38627.0,EIM IS Equity
2,2008-01-03,425.85,320648.0,EIM IS Equity
3,2008-01-04,415.50,72443.0,EIM IS Equity
4,2008-01-07,414.85,93726.0,EIM IS Equity


In [ ]:
df1=df.copy()
start=datetime.datetime(2008, 1, 1)
end=datetime.datetime(2019, 11, 30)
df1 = df1[df1['date']>=start]
df1 = df1[df1['date']<=end]
df1.reset_index(inplace = True)

In [ ]:
df1

,index,date,PX_LAST,PX_VOLUME,Ticker
0,0,2008-01-01,407.80,31627.0,EIM IS Equity
1,1,2008-01-02,404.10,38627.0,EIM IS Equity
2,2,2008-01-03,425.85,320648.0,EIM IS Equity
3,3,2008-01-04,415.50,72443.0,EIM IS Equity
4,4,2008-01-07,414.85,93726.0,EIM IS Equity
...,...,...,...,...,...
144669,144669,2019-11-25,63.75,171923335.0,YES IS Equity
144670,144670,2019-11-26,63.15,399791670.0,YES IS Equity
144671,144671,2019-11-27,68.25,254612192.0,YES IS Equity
144672,144672,2019-11-28,70.05,223021365.0,YES IS Equity


In [ ]:
df1.drop('index',axis = 1,inplace = True)
df1.drop('PX_VOLUME',axis = 1,inplace = True)

In [ ]:
len(df1['Ticker'].unique())

50

In [ ]:
len(df1['Ticker'].unique())

50

In [ ]:
#Removing those companies which have not traded for all the days in the 2008 to 2019 period.
df_s=df1.groupby('Ticker').count()
df_s.reset_index(inplace=True)

max = 0
for i in range(len(df_s)):
    if df_s.iloc[i]['date']>max:
        max = df_s.iloc[i]['date']
tickers_not = []
for j in range(len(df_s)):
    if df_s.iloc[j]['date']!= max:
        tickers_not.append(df_s.iloc[j]['Ticker'])
    
    


In [ ]:
for ticker in tickers_not:
    df1 = df1[df1['Ticker']!= ticker]

In [ ]:
start_date = pd.date_range(start = '2008-01-01',end = '2019-11-30',freq ='MS').strftime("%Y-%b-%d").to_list()
for i in range (len(start_date)):
    start_date[i]= pd.to_datetime(start_date[i],dayfirst=True)

start_date

In [ ]:
#make rolling time period of time = 1 year to account for seasonal changes in prices and do this in a loop
final_pairs={}
for i in range (len(start_date)):
    
    df_dummy=df1.copy()
    
    #take start and end date for pair forming period
    
    if i <12:
        continue
    else:
        start = start_date[i-12]
        end = start_date[i]
    date=[]

    
    print("Start dates taken!")
    
    #remove all dates other than pair forming period
    
    df_dummy = df_dummy[df_dummy['date']>=start]
    df_dummy = df_dummy[df_dummy['date']<end]
    print('Other dates removed from df_dummy')
    
    #some necessary steps
    tickers=df_dummy['Ticker'].unique().tolist()
    
    date_list=[]
    
    df_forDate=df_dummy.groupby('date').count()
    df_forDate.reset_index(inplace=True)
    for k in range (len(df_forDate)):
        date_list.append(df_forDate.iloc[k]['date'])
    b=[]
    h=0
    for ticker in tickers:
        a=[]
        for k in range (len(df_dummy)):
            #print(k)
            if df_dummy.iloc[k]['Ticker']==ticker:
                a.append(df_dummy.iloc[k]['PX_LAST'])
        b.append(a)  
    df_dummy2=pd.DataFrame(b,columns=date_list)
    print('df2_dummy formed, now calculating p_score')
    
    #p_score calculations
    warnings.filterwarnings("ignore")
    
    c=[]
    for k in range (len(df_dummy2)):
        d=[]
        for j in range (len(df_dummy2)):
            score,pvalue,_ =coint(df_dummy2.iloc[k][:],df_dummy2.iloc[j][:])
            d.append(pvalue)
        c.append(d)
    df_dummy3=pd.DataFrame(c,columns=tickers)
    
    print('pscore calculation done and stored in df_dummy3')
    
    df_dummy3["ticker"]=tickers
    df_dummy3.set_index('ticker',inplace=True)
    threshold = 0.01
    df_dummy4=df_dummy3[df_dummy3[:][:]<threshold]
    df_dummy4.reset_index(inplace=True)
    df_dummy4.drop("ticker",inplace=True,axis=1)
    
    print('now forming pairs based on threshold!')
    
    pairs={}
    for k in range(len(df_dummy4)):
        x=[]
        for j in range (len(tickers)):
            if df_dummy4.iloc[k][tickers[j]]>0:
                x.append(tickers[j])
        pairs[tickers[k]]=x
    
    print('pairs dictionary formed, now removing duplicates')
    
    empty_keys = [k for k,v in pairs.items() if not v]
    for k in empty_keys:
        del pairs[k]
        
    final=[]
    for key in pairs:
        for k in range (len(pairs[key])):
            y = []
            y.append(key)
            y.append(pairs[key][k])
            #t.append(y)
        final.append(y)
    
    rep=[]
    for k in range(len(final)):
        for j in range (k+1,len(final)):
            if final[k][0]==final[j][1]:
                if final[k][1]==final[j][0]:
                    rep.append(j)
    for index in sorted(rep, reverse=True):
        del final[index]
        
    final_pairs[start]=final
    
    
    del df_dummy
    del df_dummy2
    del df_dummy3
    del df_dummy4
    del df_forDate
        
    

    

    
        

            
    

Start dates taken!
Other dates removed from df_dummy
df2_dummy formed, now calculating p_score
pscore calculation done and stored in df_dummy3
now forming pairs based on threshold!
pairs dictionary formed, now removing duplicates
Start dates taken!
Other dates removed from df_dummy
df2_dummy formed, now calculating p_score
pscore calculation done and stored in df_dummy3
now forming pairs based on threshold!
pairs dictionary formed, now removing duplicates
Start dates taken!
Other dates removed from df_dummy
df2_dummy formed, now calculating p_score
pscore calculation done and stored in df_dummy3
now forming pairs based on threshold!
pairs dictionary formed, now removing duplicates
Start dates taken!
Other dates removed from df_dummy
df2_dummy formed, now calculating p_score
pscore calculation done and stored in df_dummy3
now forming pairs based on threshold!
pairs dictionary formed, now removing duplicates
Start dates taken!
Other dates removed from df_dummy
df2_dummy formed, now calcu

In [ ]:
#final_pairs

In [ ]:
start_date=pd.date_range('2008-01-01','2019-11-30', freq='MS').strftime("%Y-%b-%d").tolist()
df['date']= pd.to_datetime(df['date'],dayfirst=True)
start_date = pd.to_datetime(start_date,dayfirst=True)

In [ ]:
#for buy  - you buy expensive stock,sell cheap stock
#for sell = sell expensive stock,buy cheap stock
def returns (pair, st_date,end_date,amount):
    y=amount
    df1=df.copy()
    df1 = df1[df1['Ticker']==pair[0]]
    df1 = df1[df1['date']>=st_date]
    df1 = df1[df1['date']<end_date]
    df2=df.copy()
    df2 = df2[df2['Ticker']==pair[1]]
    df2 = df2[df2['date']>=st_date]
    df2 = df2[df2['date']<end_date]
    date = df1['date'].tolist()
    pr1 = df1['PX_LAST'].tolist()
    pr2 = df2['PX_LAST'].tolist()

    main_df = pd.DataFrame(
        {'date': date,
         pair[0]: pr1,
         pair[1]: pr2})
    
    ratio = []
    for i in range(len(main_df)):
        ratio.append(math.log(main_df.iloc[i][pair[1]]/main_df.iloc[i][pair[0]]))
    main_df['ln']=ratio
    
    
    mvg_avg=[]
    std_dev=[]
    main_df['mvg_avg'] = main_df['ln'].rolling(window = 10).mean()
    main_df['std_dev'] = main_df['ln'].rolling(window = 10).std()
    
    z_score=[]
    for i in range(len(main_df)):
        z_score.append((main_df.iloc[i]['ln']-main_df.iloc[i]['mvg_avg'])/main_df.iloc[i]['std_dev'])
    main_df['z_score']=z_score
    main_df.fillna(0,axis = 0,inplace = True)
    
    buy_threshold=-2
    sell_threshold=2
    SL=4
    TP=0
    
    buy_threshold=-2
    sell_threshold=2
    SL=4
    TP=0
    
    signal=[]
    for i in range(len(main_df)):
        if (main_df.iloc[i]['z_score']<buy_threshold) & (abs(main_df.iloc[i]['z_score'])<SL):
            signal.append('BUY')
        elif (main_df.iloc[i]['z_score']>sell_threshold) & (abs(main_df.iloc[i]['z_score'])<SL):
            signal.append('SELL')
        else:
            signal.append('')
    main_df['signal']=signal
    
    
    status=['']*len(main_df)
    for i in range (len(main_df)):
        #print (i)
        if status[i-1]=='' or status[i-1]=='SL' or status[i-1]=='TP':
            status[i]= main_df.iloc[i]['signal']
        elif abs(z_score[i])> SL:
            if status[i-1]=='BUY' or status[i-1]=='SELL':
                status[i]= 'SL'
        elif z_score[i-1]/z_score[i]<0:
            if status[i-1]=='BUY' or status[i-1]=='SELL':
                status[i]= 'TP'
        else:
            status[i]=status[i-1]
            
    main_df['status']=status
    pos1_open=0
    pos1_close=0
    pos2_open=0
    pos2_close=0
    position=0
    profit=[]
    pos1_pnl=0
    pos2_pnl=0
            
    for i in range (len(main_df)):
        
        if pos1_open==0:
            if status[i]=='BUY':
                pos1_open=main_df.iloc[i][pair[1]]
                pos2_open=main_df.iloc[i][pair[0]]
                position = 1
                #print (pos1_open)
            if status[i]=='SELL':
                pos1_open=main_df.iloc[i][pair[1]]
                pos2_open=main_df.iloc[i][pair[0]]
                position = -1
                #print (pos1_open)
        if status[i]=='SL' or status[i]=='TP':
            pos1_close= main_df.iloc[i][pair[1]]
            pos2_close= main_df.iloc[i][pair[0]]
            if position ==-1:
                pos1_pnl=(pos1_open-pos1_close)*100/pos1_open
                pos2_pnl=(pos2_close-pos2_open)*100/pos2_open
            if position ==1:
                pos2_pnl=(pos2_open-pos2_close)*100/pos2_open
                pos1_pnl=(pos1_close-pos1_open)*100/pos1_open
            pos1_open=0
            pos1_close=0
            pos2_open=0
            pos2_close=0
            position=0
            profit.append(pos1_pnl+pos2_pnl)
    if position!=0:
        main_df.at[len(main_df)-1, 'status']='Square Off'
        pos1_close=main_df.iloc[len(main_df)-1][pair[1]]
        pos2_close=main_df.iloc[len(main_df)-1][pair[0]]
        if position==1:
            pos2_pnl=(pos2_open-pos2_close)*100/pos2_open
            pos1_pnl=(pos1_close-pos1_open)*100/pos1_open
        if position ==-1:
            pos1_pnl=(pos1_open-pos1_close)*100/pos1_open
            pos2_pnl=(pos2_close-pos2_open)*100/pos2_open
        #print (pos1_close)
        #print (pos2_close)
        pos1_open=0
        pos1_close=0
        pos2_open=0
        pos2_close=0
        position=0
        profit.append(pos1_pnl+pos2_pnl)
        
    
    for i in range (len(profit)):
        y = y*(1+profit[i]/100)
        #print (y)
    return sum(profit), profit, main_df, y

In [ ]:
st_date = datetime.datetime(2019, 2, 1)
end_date = datetime.datetime(2019, 5, 1)
invested_amount = 1000
pair = ['RIL IS Equity', 'INFO IS Equity']
net_profit, profit_by_trades, history, amount=returns(pair, st_date,end_date,invested_amount)
trading_sheet = history.copy()
trading_sheet.drop(['ln','mvg_avg','std_dev'],inplace=True,axis=1)
trading_sheet.set_index('date',inplace=True)

In [ ]:
print('The pair involves\033[1m',pair[0],'\033[0mand\033[1m',pair[1],'\033[0mover the trading period\033[1m',st_date,'\033[0m to \033[1m',end_date,'\033[0m')
print('The net profit over the trading period is\033[1m',round(net_profit,ndigits=2),'%\033[0m')
print('If\033[1m',round(invested_amount,ndigits=2),'\033[0mINR is invested over the trading period (with compounding), it finally amounts to\033[1m',round(amount,ndigits=2),'\033[0m INR')
print('The respective profit on each individual position is\033[1m',list(np.around(np.array(profit_by_trades),2)),'\033[0m in %')
print('The trading sheet over the period is:-')
print(tabulate(trading_sheet, headers='keys', tablefmt='psql'))

The pair involves RIL IS Equity and INFO IS Equity over the trading period 2019-02-01 00:00:00  to  2019-05-01 00:00:00 
The net profit over the trading period is 8.81 %
If 1000 INR is invested over the trading period (with compounding), it finally amounts to 1090.03  INR
The respective profit on each individual position is [4.52, 4.29]  in %
The trading sheet over the period is:-
+---------------------+-----------------+------------------+-------------+----------+----------+
| date                |   RIL IS Equity |   INFO IS Equity |     z_score | signal   | status   |
|---------------------+-----------------+------------------+-------------+----------+----------|
| 2019-02-01 00:00:00 |         1249.95 |           757.05 |  0          |          |          |
| 2019-02-04 00:00:00 |         1290.9  |           755.9  |  0          |          |          |
| 2019-02-05 00:00:00 |         1291.55 |           754.85 |  0          |          |          |
| 2019-02-06 00:00:00 |         13

In [ ]:
def portfolio_returns (st_date,end_date,amount):
    #pair_date = datetime.datetime()
    pair_list = []
    #print(type(start_date))
    for i in range(len(start_date)):
        if st_date==start_date[i]:
            #print(st_date)
            #print(start_date[i])
            pair_date = start_date[i-12]
            #print(pair_date)
    for time in final_pairs:
        if pair_date==time:
            pair_list=final_pairs[time]
    return_list=[]
    final_amount_list = []
    no_pairs=0
    for pair in pair_list:
        no_pairs = no_pairs+1
        net_profit, profit_by_trades, history, final_amount = returns(pair, st_date,end_date,invested_amount)
        return_list.append(net_profit)
        final_amount_list.append(final_amount)
    portfolio_return = sum(return_list)
    total_amount = sum(final_amount_list)
    
    return portfolio_return, total_amount, no_pairs,return_list

In [ ]:
st_date = datetime.datetime(2018, 9, 1)
end_date = datetime.datetime(2018, 12, 1)
amount_invested=1000
per_portfolio_return, final_amount, no_pairs = portfolio_returns (st_date,end_date,amount_invested)